# Secret Manager
Secret Manager is a secure and convenient storage system for API keys, passwords, certificates, and other sensitive data. 

You should **NEVER** write a sensitive data inside your code. Instead, you can create a secret inside GCP and read the secret inside your code. By doing so, anybody, even with your code, can get acces to the secret. Only your application get the secret value.

## Create a secret

To create a secret, we're gonna use the gcloud (interact with GCP on a terminal).
First, we create a file with our secret value inside. We do that as test but, for reminder, you should never write a secret value inside the code.

In [1]:
# Creating a file with a secret inside
file = open("my_little_secret.txt", "w") 
file.write("This is my secret, keep it hidden") 
file.close()

In [2]:
# get your active directory path
import os 
os.path.abspath(os.getcwd())

'/Users/garantme/PycharmProjects/someTuto/gcp'

Now, we can use gcloud to create the secret. 
- Replace "your_path" by the path you get form the previous cell
- Replace "secret-id" by a name of your choice. This will be the name of your secret

In [3]:
! gcloud secrets create secret-id \
--replication-policy="user-managed" \
--locations="europe-west3" \
--data-file="your_path/my_little_secret.txt"

Created version [1] of the secret [secret-id].


## Read a secret inside an application

### open impersonated session
First, we will authenticate ourselves as service account in gcp. Why ? Your future application will be execute by a service account. We put ourselves as a real situation.

In [4]:
! gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Cgx1ZY5sciaXK9cYoCTADlykrNWa8D&access_type=offline&code_challenge=ej-J-VGUq2-fHJkRrlQo3dPtsspURHPp8c8lNpFRQh4&code_challenge_method=S256


Credentials saved to file: [/Users/garantme/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Quota project "datawarehouse-267911" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resou

In [5]:
# We are using the secretmanager lib
from google.cloud import secretmanager

# open the secret manager client
client = secretmanager.SecretManagerServiceClient()
secret_id = input("Your secret name you have defined : ")
name = f"projects/datawarehouse-267911/secrets/{secret_id}/versions/latest"
# request the secret
response = client.access_secret_version(request={"name": name})


# We rare printing the secret value here to show you how it's working
# In a real case, do not do that
payload = response.payload.data.decode("UTF-8")
print("Plaintext: {}".format(payload))

Your secret name you have defined : secret-id
Plaintext: This is my secret, keep it hidden


### Secret version
A secret may have multiple version. If you pay attention on the previous request, we specify inside the variable name, "version/latest". 

Now a little exercice, add a secret version to your secret and then read the first version of the secret.
The documentation is here: https://cloud.google.com/secret-manager/docs/creating-and-accessing-secrets

In [6]:
import hide_cell
hide_cell.hide_toggle(True)

In [8]:
# Creating a file with a secret inside
file = open("my_little_secret.txt", "w") 
file.write("This is my new secret, way diffrent from the first one") 
file.close()

# change the name of your secret here
# and change the path of your file
! gcloud secrets versions add secret-id \
--data-file="your_path/my_little_secret.txt"

name = f"projects/datawarehouse-267911/secrets/{secret_id}/versions/1"
# request the secret
response = client.access_secret_version(request={"name": name})

# We rare printing the secret value here to show you how it's working
# In a real case, do not do that
payload = response.payload.data.decode("UTF-8")
print("Plaintext: {}".format(payload))

Created version [2] of the secret [secret-id].
Plaintext: This is my secret, keep it hidden


Après ce tutoriel, pensez à supprimer le secret que vous créé

In [9]:
name = client.secret_path("datawarehouse-267911", "secret-id")
client.delete_secret(request={"name": name})